In [1]:
import requests
from pymongo import MongoClient
from urllib.parse import urljoin
from pprint import pprint
import nltk
import json

# Retrieve Lyrics Via API

In [2]:
# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db
songs_collection = db.top_songs

In [3]:
# Pull all songs from DB
songs = list(songs_collection.find())

In [4]:
MUSICMATCH_API = '309401c7a9740d7c3bfbc94bca9a911c'
APISEED_API = 'ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0'

In [5]:
class ApiSeed:
    BASE_URL = 'https://orion.apiseeds.com/api/music/lyric/'
    
    def __init__(self, api_key):
        """ Load automatically translates responses from HTTP request objects into Dictionaries """
        self.api_key = api_key
        self.load = True
    
    def _api_url(self, url):
        url = urljoin(self.BASE_URL, url)
        url = urljoin(url, '?apikey=ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0')
        return url
    
    def _retrieve(self, url):
        response = requests.get(self._api_url(url))
        if self.load:
            response = json.loads(response.content)
        return response
    
    def get_track(self, artist, track):
        return self._retrieve(f'{artist}/{track}')
    

apiseed = ApiSeed(APISEED_API)
res = apiseed.get_track('Kanye West', 'Everything I am')

# Insert Lyrics into DB

In [6]:
import time
def insert_into_db():
    for song in songs:
        pass
    
def update_db(document_id, lyrics: str):
    document = songs_collection.find_one_and_update({"_id": document_id}, 
                                         {"$set": {"lyrics": lyrics}})
    return document

def retrieve_lyrics(track: dict):
    """ Takes mongodb document and gets lyrics"""
    res = apiseed.get_track(track['artist'], track['title'])
    return res

def find_and_update_lyrics(track: dict):
    """ Updates track with lyrics """
    lyrics = retrieve_lyrics(track)
    track['lyrics'] = lyrics
    return track

        
start_index = 0
all_tracks = []
for i, track in enumerate(songs[start_index:], start_index):
    try:
        track = find_and_update_lyrics(track)

        if track.get('lyrics').get('error'):
            print("Error", end=" "); 
        else:
            print("Success", end=" ")
            
    except (json.JSONDecodeError, Exception) as e:
        print(e)
        track['lyrics'] = 'null'
        
    update_db(track['_id'], track['lyrics'])
    print(i, track['title'], end = " | ")
    time.sleep(.33)



Success 0 In My Feelings | Error 1 Girls Like You | Error 2 I Like It | Error 3 FEFE | Success 4 Better Now | Success 5 Lucid Dreams | Error 6 Sicko Mode | Error 7 Taste | Success 8 Boo'd Up | Error 9 No Brainer | Error 10 Love Lies | Success 11 No Tears Left To Cry | Error 12 Psycho | Error 13 Nice For What | Error 14 Yes Indeed | Success 15 Delicate | Success 16 Youngblood | Success 17 Barbie Dreams | Error 18 Meant To Be | Success 19 Perfect | Success 20 Nonstop | Error 21 The Middle | Error 22 Big Bank | Success 23 God's Plan | Success 24 Back To You | Error 25 Sad! | Success 26 Stargazing | Success 27 Tequila | Success 28 Mercy | Success 29 God Is A Woman | Error 30 Mine | Error 31 One Kiss | Error 32 Friends | Success 33 Natural | Error 34 Simple | Success 35 Drowns The Whiskey | Success 36 Never Be The Same | Success 37 In My Blood | Success 38 I Like Me Better | Success 39 Get Along | Error 40 Apes**t | Error 41 Bed | Success 42 Yosemite | Success 43 Chun-Li | Success 44 Life C

In [163]:
songs_collection = db.top_songs

In [164]:
songs = list(songs_collection.find())

In [172]:
len(songs)

7354